# Preprocess chain from Raster/SHP to input format of the net

Requirements can be found in requirements.txt
This code relies on GDAL


Set the directory path for the project

For DevLab team: must be run from `~/ekna_kiln_detect`

In [9]:
#make dir where the root folder of the project is
# %env DIR=~/ekna_kiln_detect
!pwd

/home3/idavis1/ekna_kiln_detect


In [10]:
%%bash

# remove BROWSE files if they exist
find . -name '*BROWSE*' -delete

Create the tiles to run the model

The output images goes to inputs/Tiled
Variables to set:
* SIZE  size in pixels of the tile cut
* JOBS number of parallel jobs desired to run this code
* OVERLAP number of pixels you want the tiles to overlap. Useful to later handle better the objects detected on the edges

In [3]:
!nproc

2


In [ ]:
# !mkdir ~/ekna_kiln_detect/test_2024/images/
# !cp ~/ekna_kiln_detect/imagery/test_image/test_R4C2.tif ~/ekna_kiln_detect/test_2024/images/

mkdir: cannot create directory ‘/home3/jackjaco/ekna_kiln_detect/test_2024/images/’: File exists


#### This cell creates a different image processing command for each of the subimages

In [13]:
%%bash

SIZE=512  # Lowered from 1024 on Laura's recommendation
#JOBS=36 -- overwriting this because running `!nproc` gives value of 2
JOBS=2
OVERLAP=20
mkdir -p scripts_toerase
mkdir -p inputs/Tiled
chmod +x VerhegghenReplica/scripts/processing/*.sh
for file in /data4/shared/ekna_kiln_drive/images/test2_pancro/*tif   # Previously: /data4/shared/ekna_kiln_drive/images/test1/*tif
do
    echo VerhegghenReplica/scripts/processing/maketiles.sh ${file} $SIZE $(basename "$file" .tif) $OVERLAP inputs/Tiled/ > scripts_toerase/'mt_'$(basename "$file")'.sh'
done

pwd


/home3/idavis1/ekna_kiln_detect


#### This tiles the images

In [14]:
%%bash

# Make sure the tiling script is executable
chmod +x scripts_toerase/mt_*.sh

# Add all script execution calls to a single text file and print the number of lines
ls scripts_toerase/mt_* > scripts_toerase/Main-par.txt
ls scripts_toerase/mt_* | wc -l

# Send the tiling commands into a parallel job
cat scripts_toerase/Main-par.txt | ~/miniconda3/envs/kiln/bin/parallel -j 2 {}

# Remove the intermediate script directory and its contents
rm -r  scripts_toerase

1
-------------------------------------------------------------/data4/shared/ekna_kiln_drive/images/test2_pancro/test_pancro1_R2C3.tif
image file is /data4/shared/ekna_kiln_drive/images/test2_pancro/test_pancro1_R2C3.tif
desired tile size is  512 x 512
image is  4463 columns by  16384  rows
prefix is test_pancro1_R2C3
overlap is 20 pixels.

Estimating 9 columns of tiles (X), by 33 rows of tiles (Y).

Taking a closer look at columns.
When laying out 9 columns of tiles, it looks like the final column will fall at 4448 pixels.
That won't even make it to the end of the image! The overlap requires creating one more column of tiles than I had estimated.
Trying now with 10 columns.
When laying out 10 columns of tiles, it looks like the final column will fall at 4940 pixels.
That's bigger than the image, so we'll back off the final column of tiles to begin at 3951 and end exactly at 4463.
This final column of tiles will overlap the second-to-last column by 497 pixels.

Taking a closer look at 

In [ ]:
%%bash

pwd
find inputs/Tiled -maxdepth 1 -name "*tif" > inputs/list_tiles.csv   # Added -maxdepth 1 to avoid prior runs' tiles in /RGB or /pancro being included
mkdir -p inputs/Tiled/pancro inputs/Tiled/RGB

/home3/idavis1/ekna_kiln_detect


### Erase tiles that are all 0
Also separate tiles that belong to RGB and Pancro in different folders

in this case it creates a compilation of directories of 100 images under each type. To ensure computation


In [17]:
%%bash

pwd
python3 VerhegghenReplica/scripts/processing/erase_tiles_nodata.py

/home3/idavis1/ekna_kiln_detect


/home3/idavis1
38
Finish!!! :). Execution time: 1.1092913150787354
inputs/Tiled/test_pancro1_R2C3_26_1.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_11_10.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_1_4.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_18_9.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_27_6.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_2_6.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_15_1.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_12_2.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_31_9.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_11_8.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_2_2.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_10_2.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_23_4.tif
move it to pancro folder
inputs/Tiled/test_pancro1_R2C3_26_3.tif
move it to pancro folder
inputs/Tiled/test_pancro1

In [18]:
%%bash

# CLEAN THE tILES FOLDERS FROM *XML AND *tfw
rm inputs/Tiled/*xml
rm inputs/Tiled/*tfw

In [ ]:
## JJ - DO NOT RUN - this was written to get around limits in files per subdirectory on their system
# %%bash

## Create subfolders of 100 elements each to be able to handle them
# cd inputs/Tiled/pancro
# i=0; for f in *; do d=dir_$(printf %03d $((i/100+1))); mkdir -p $d; mv "$f" $d; let i++; done
# cd inputs/Tiled/RGB
# i=0; for f in *; do d=dir_$(printf %03d $((i/100+1))); mkdir -p $d; mv "$f" $d; let i++; done


Create the footprints of the tiles, one for RBG and another one for the pancromatic images

- Create the 'footprints': associate the image objects with a geographic footprint and check that it covers the area
- Check that we're able to use EPSG:4326 (or whatever our Maxar endpoint uses)

In [19]:
%%bash

rm -r inputs/footprints
mkdir -p inputs/footprints

#create the footprints for RGB and pancro images
find inputs/Tiled/pancro -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-pancro.shp 
find inputs/Tiled/RGB -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-RGB.shp

find inputs/Tiled/pancro -type f -maxdepth 1 -name "*tif" > inputs/list_tiles_pancro.csv
find inputs/Tiled/RGB -type f -maxdepth 1 -name "*tif" > inputs/list_tiles_RGB.csv

Creating new index file...
Creating new index file...


# DevLab Team: YOU DON'T NEED THIS PART ON EARLY RUNS

### Run the preprocess algorithm to create the dataset to train/validate/test

In this case we will need the shapefile where the ground truth data is delineated, in our case is in DIR/extended_AOI/
Parameters:
- CSV with the list of images
- Where to output the image. The code handles the distirbution of pancro and RGB
- Folder to put in images with annotations
- Folder to put in images without annotations
- The name of the shapefile with the objects to train
- Name and path of the json file where the annotations will be saved

In [ ]:
%%bash

mkdir -p inputs/pancro/img_with_ann/  inputs/RGB/img_with_ann/ inputs/pancro/img_without_ann/  inputs/RGB/img_without_ann/ inputs/pancro/annotations/  inputs/RGB/annotations/
# Need new shapefile:  extended_AOI/ekna_test_aoi_20241206.shp --> ??
# Uncommenting next line for pancro
python3 VerhegghenReplica/scripts/processing/image_preprocess.py inputs/list_tiles_pancro.csv inputs/pancro/img_with_ann/ inputs/pancro/img_without_ann/ extended_AOI/ekna_test_aoi_20241206.shp inputs/pancro/annotations/pancro_all.json
python3 VerhegghenReplica/scripts/processing/image_preprocess.py inputs/list_tiles_RGB.csv inputs/RGB/img_with_ann/ inputs/RGB/img_without_ann/ extended_AOI/ekna_test_aoi_20241206.shp inputs/RGB/annotations/RGB_all.json
pwd

Traceback (most recent call last):
  File "/home3/idavis1/ekna_kiln_detect/VerhegghenReplica/scripts/processing/image_preprocess.py", line 175, in <module>
    main()
  File "/home3/idavis1/ekna_kiln_detect/VerhegghenReplica/scripts/processing/image_preprocess.py", line 167, in main
    img_id, annotations_id, images, annotations = preprocessshape(f, img_id, annotations_id, images, annotations)
                                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home3/idavis1/ekna_kiln_detect/VerhegghenReplica/scripts/processing/image_preprocess.py", line 79, in preprocessshape
    layer, driver, dataSource = shapefile.openshp(shpname, 0)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home3/idavis1/ekna_kiln_detect/VerhegghenReplica/scripts/GDAL-python/shapefile.py", line 31, in openshp
    layer = dataSource.GetLayer()
            ^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'GetLay

/home3/idavis1/ekna_kiln_detect


Create the new footprints, one for images with annotations and another one for images without

In [ ]:
%%bash

# pancro images
path_pancro_ann=inputs/pancro/img_with_ann/
path_pancro_noann=inputs/pancro/img_without_ann/ 

find $path_pancro_noann -type f -maxdepth 1 -name "*tif" > inputs/pancro_listwithoutann.csv
find $path_pancro_ann  -type f -maxdepth 1 -name "*tif" > inputs/pancro_listwithann.csv

find $path_pancro_noann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-pancro-noann.shp 
find $path_pancro_ann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-pancro-ann.shp

# RGB images
path_RGB_ann=inputs/RGB/img_with_ann/
path_RGB_noann=inputs/RGB/img_without_ann/ 

find $path_RGB_noann -type f -maxdepth 1 -name "*tif" > inputs/RGB_listwithoutann.csv
find $path_RGB_ann  -type f -maxdepth 1 -name "*tif" > inputs/RGB_listwithann.csv

find $path_RGB_noann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-RGB-noann.shp 
find $path_RGB_ann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  inputs/footprints/footprint-RGB-ann.shp

Creating new index file...
Creating new index file...
Creating new index file...
Creating new index file...


In [46]:
%%bash
#change paths inside the files
sed -i "s|/scratch/swalim/inputs/RGB/img_with_ann|$DIR/inputs/RGB/img_with_ann|g" $DIR/inputs/RGB_listwithann.csv
sed -i "s|/scratch/swalim/inputs/pancro/img_with_ann|$DIR/inputs/pancro/img_with_ann|g" $DIR/inputs/pancro_listwithann.csv

sed -i "s|/scratch/swalim/inputs/RGB/img_without_ann|$DIR/inputs/RGB/img_without_ann|g" $DIR/inputs/RGB_listwithoutann.csv
sed -i "s|/scratch/swalim/inputs/pancro/img_without_ann|$DIR/inputs/pancro/img_without_ann|g" $DIR/inputs/pancro_listwithoutann.csv

